<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=f73eb51b59c2b7f6166a0d8f996cb1035010292165e2906dc3cf97d61e096715
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-08 15:43:59
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.56 C
-------------------
Today PnL: -2.43 L (-1.54%)
Current PnL: -27.22 L (-17.36%)
CY Booked + Current PnL: -12.93 L (-8.25%)
-------------------
Total profit:  1.77 L
Total loss:  -28.98 L
-------------------
Total Booked + Current PnL: 14.05 L (10.82%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.16%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.52 C
Est FTT PnL: 96.23 L (61.72%)
Deployed:  1.30 C
Current:  1.56 C
CAGR/XIRR %: 10.16%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.60,15.22,5.36,21.40,16392.0,40389.0,305815.0,6.96,64.0,X-LC,5.54,14.0,2.46,1.96,22.51,XY25,NTT,FMCG
80,TTKPRESTIG,770.00,-1.01,-21.40,27.35,0.09,21664.0,-21568.0,79209.0,83.78,35.0,M-SC,3.24,253.0,-1.00,0.51,4.06,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-1.48,4.57,14.01,19.23,22044.0,6882.0,157342.0,22.81,74.0,L-LC,6.58,182.0,0.31,1.01,26.82,XY25,ATH,MINING
39,INDIGOPNTS,1408.00,-0.07,-13.26,15.32,0.02,23195.0,-23155.0,151404.0,131.03,58.0,M-SC,7.79,240.0,-1.00,0.97,31.70,OX40N,NTT,PAINTS
49,MASFIN,398.61,-4.15,-1.64,24.08,22.05,23207.0,-1605.0,96375.0,-15.09,51.0,H-SC,6.59,168.0,-0.07,0.62,40.09,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,4546.37,4.50,-31.54,182.82,93.63,126369.0,-31841.0,69122.0,-26.73,48.0,H-SC,16.52,143.0,-0.25,0.44,4.50,SR,ATH,CHEMICALS
79,TRIDENT,48.00,3.32,-27.42,77.06,28.51,51601.0,-25292.0,66962.0,-14.17,50.0,M-SC,7.18,226.0,-0.49,0.43,14.58,XR,NTT,TEXTILES
22,DIXON,18931.72,1.84,-16.89,57.94,31.26,111124.0,-38976.0,191792.0,-61.22,35.0,X-MC,18.44,56.0,-0.35,1.23,2.23,X40N,ATH,IT
44,JCHAC,2282.00,0.75,-34.84,53.45,-0.01,44512.0,-44527.0,83278.0,16943.48,48.0,M-SC,13.90,235.0,-1.00,0.53,6.59,OX40N,BTT,AC
71,STARHEALTH,761.00,0.70,-13.38,70.00,47.26,170465.0,-37601.0,243522.0,18.32,39.0,H-SC,7.71,174.0,-0.22,1.56,30.47,XY24,NTT,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-5.25,-7.88,58.02,45.57,139833.0,-20611.0,241008.0,-40.82,40.0,H-MC,4.66,121.0,-0.15,1.55,2.55,AR,NTT,ELECTRICAL
56,RAJOOENG,143.10,-4.17,-29.86,99.05,39.61,71207.0,-30610.0,71890.0,-52.57,56.0,H-SC,31.25,136.0,-0.43,0.46,16.36,AR,ATH,MISC
49,MASFIN,398.61,-4.15,-1.64,24.08,22.05,23207.0,-1605.0,96375.0,-15.09,51.0,H-SC,6.59,168.0,-0.07,0.62,40.09,XR,ATH,FINANCE
68,SIEMENS,4671.50,-3.96,-19.14,55.23,25.51,83107.0,-35620.0,150475.0,-7.73,37.0,H-LC,1.63,50.0,-0.43,0.97,10.89,AR,ATH,ELECTRICAL
57,RECLTD,446.00,-3.60,0.54,20.09,20.74,41038.0,1100.0,204270.0,33.49,59.0,M-MC,5.93,189.0,0.03,1.31,11.21,XY25,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,-3.6,0.54,20.09,20.74,41038.0,1100.0,204270.0,33.49,59.0,M-MC,5.93,189.0,0.03,1.31,11.21,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.88,-1.75,111.62,107.91,161912.0,-2588.0,145056.0,-20.15,53.0,M-SC,10.19,220.0,-0.02,0.93,8.16,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.07,-13.26,15.32,0.02,23195.0,-23155.0,151404.0,131.03,58.0,M-SC,7.79,240.0,-1.00,0.97,31.70,OX40N,NTT,PAINTS
75,TATAELXSI,9161.00,-2.33,-17.61,60.26,32.03,65451.0,-23221.0,108614.0,-10.83,66.0,H-SC,6.96,157.0,-0.35,0.70,21.66,OX40N,NTT,IT
80,TTKPRESTIG,770.00,-1.01,-21.40,27.35,0.09,21664.0,-21568.0,79209.0,83.78,35.0,M-SC,3.24,253.0,-1.00,0.51,4.06,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.37,-21.71,44.93,13.47,94861.0,-58536.0,211131.0,-67.73,50.0,H-SC,4.79,159.0,-0.62,1.35,7.35,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,-3.60,0.54,20.09,20.74,41038.0,1100.0,204270.0,33.49,59.0,M-MC,5.93,189.0,0.03,1.31,11.21,XY25,NTT,FINANCE
17,COALINDIA,484.83,-1.48,4.57,14.01,19.23,22044.0,6882.0,157342.0,22.81,74.0,L-LC,6.58,182.0,0.31,1.01,26.82,XY25,ATH,MINING
87,WIPRO,420.00,-3.18,7.54,60.18,72.27,109665.0,12781.0,182229.0,-7.89,50.0,M-LC,6.14,101.0,0.12,1.17,14.18,XR,NTT,IT
37,IEX,219.00,-3.03,1.86,45.91,48.63,93506.0,3718.0,203672.0,-32.21,63.0,H-SC,13.59,137.0,0.04,1.31,13.70,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,WIPRO,420.00,-3.18,7.54,60.18,72.27,109665.0,12781.0,182229.0,-7.89,50.0,M-LC,6.14,101.0,0.12,1.17,14.18,XR,NTT,IT
37,IEX,219.00,-3.03,1.86,45.91,48.63,93506.0,3718.0,203672.0,-32.21,63.0,H-SC,13.59,137.0,0.04,1.31,13.70,XR,NTT,MISC
25,FINCABLES,1641.55,-0.88,-1.75,111.62,107.91,161912.0,-2588.0,145056.0,-20.15,53.0,M-SC,10.19,220.0,-0.02,0.93,8.16,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.09,-6.88,122.00,106.72,140112.0,-8490.0,114846.0,-53.87,37.0,H-SC,7.21,138.0,-0.06,0.74,14.88,AR,ATH,MISC
49,MASFIN,398.61,-4.15,-1.64,24.08,22.05,23207.0,-1605.0,96375.0,-15.09,51.0,H-SC,6.59,168.0,-0.07,0.62,40.09,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.10,-16.45,32.59,10.78,65437.0,-39528.0,200790.0,-50.14,13.0,X-LC,5.77,1.0,-0.60,1.29,0.00,X40,NTT,FMCG
7,AWL,485.00,-0.52,-26.15,112.86,57.19,271809.0,-85290.0,240837.0,-65.03,27.0,X-MC,8.81,58.0,-0.31,1.54,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.08,-1.74,-12.01,48.44,30.62,101041.0,-28464.0,208590.0,-37.46,28.0,X-MC,16.18,57.0,-0.28,1.34,0.00,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-0.94,-40.19,126.07,35.20,97015.0,-51717.0,76953.0,0.94,32.0,X-SC,17.33,93.0,-0.53,0.49,0.00,X40,NTT,FOOTWEAR
59,RELIANCE,1952.00,-2.23,-5.60,32.73,25.30,124183.0,-22510.0,379415.0,-8.81,32.0,X-LC,8.44,25.0,-0.18,2.43,27.07,XY25,BTT,REFINERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UNITDSPR,1644.00,-2.00,-0.82,21.76,20.77,50534.0,-1910.0,232234.0,-9.42,34.0,X-MC,0.98,70.0,-0.04,1.49,6.05,X40N,NTT,BREWERIES
68,SIEMENS,4671.50,-3.96,-19.14,55.23,25.51,83107.0,-35620.0,150475.0,-7.73,37.0,H-LC,1.63,50.0,-0.43,0.97,10.89,AR,ATH,ELECTRICAL
20,DABUR,735.00,-0.28,1.56,41.50,43.70,104552.0,3865.0,251933.0,-6.86,64.0,X-MC,2.10,73.0,0.04,1.62,16.84,XY24,BTT,FMCG
45,JIOFIN,387.00,-3.28,-5.58,31.83,24.48,80356.0,-14912.0,252453.0,-7.18,44.0,X-LC,2.19,37.0,-0.19,1.62,46.33,XY24,BTT,FINANCE
84,VBL,671.64,-1.73,1.18,34.09,35.67,108943.0,3732.0,319574.0,-10.77,62.0,X-LC,2.19,5.0,0.03,2.05,15.20,X40N,ATH,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-1.82,-27.57,126.65,64.15,218269.0,-65611.0,172340.0,-57.62,37.0,X-SC,6.15,84.0,-0.30,1.11,0.0,XY24,BTT,CEMENT
7,AWL,485.00,-0.52,-26.15,112.86,57.19,271809.0,-85290.0,240837.0,-65.03,27.0,X-MC,8.81,58.0,-0.31,1.54,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.94,-40.19,126.07,35.20,97015.0,-51717.0,76953.0,0.94,32.0,X-SC,17.33,93.0,-0.53,0.49,0.0,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,-1.44,-22.29,82.10,41.51,168026.0,-58705.0,204660.0,-7.41,33.0,X-MC,8.96,65.0,-0.35,1.31,0.0,XY25,ATH,FMCG
52,PAGEIND,51605.08,-1.74,-12.01,48.44,30.62,101041.0,-28464.0,208590.0,-37.46,28.0,X-MC,16.18,57.0,-0.28,1.34,0.0,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-1.33,-26.53,94.42,42.84,45097.0,-17244.0,47762.0,-52.07,55.0,X-SC,27.85,83.0,-0.38,0.31,8.06,XY24,NTT,MISC
58,RELAXO,1176.00,-0.50,-48.21,194.07,52.31,145904.0,-69979.0,75181.0,-44.83,41.0,X-SC,7.74,92.0,-0.48,0.48,0.63,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.94,-40.19,126.07,35.20,97015.0,-51717.0,76953.0,0.94,32.0,X-SC,17.33,93.0,-0.53,0.49,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-3.03,0.28,23.73,24.08,30210.0,356.0,127306.0,-7.89,51.0,X-SC,6.48,91.0,0.01,0.82,19.95,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-1.09,-17.16,73.01,43.32,98505.0,-27952.0,134920.0,-28.80,46.0,X-SC,7.82,90.0,-0.28,0.87,3.59,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.10,-16.45,32.59,10.78,65437.0,-39528.0,200790.0,-50.14,13.0,X-LC,5.77,1.0,-0.60,1.29,0.00,X40,NTT,FMCG
78,TMPV,600.00,-1.07,-26.33,66.92,22.98,111372.0,-59473.0,166425.0,-23.08,46.0,X-LC,4.50,2.0,-0.53,1.07,4.67,XY24,NTT,AUTO
76,TCS,4389.97,-2.78,-11.91,37.02,20.71,123353.0,-45031.0,333206.0,-24.27,47.0,X-LC,2.59,3.0,-0.37,2.14,11.34,X40,ATH,IT
84,VBL,671.64,-1.73,1.18,34.09,35.67,108943.0,3732.0,319574.0,-10.77,62.0,X-LC,2.19,5.0,0.03,2.05,15.20,X40N,ATH,FMCG
41,INFY,1972.00,-1.57,9.83,22.23,34.24,76390.0,30745.0,343633.0,-14.57,50.0,X-LC,7.62,6.0,0.40,2.20,18.92,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-1.60,-34.78,103.75,32.89,52245.0,-26850.0,50357.0,-721.83,58.0,L-MC,7.63,259.0,-0.51,0.32,38.68,XR,NTT,BANKS
49,MASFIN,398.61,-4.15,-1.64,24.08,22.05,23207.0,-1605.0,96375.0,-15.09,51.0,H-SC,6.59,168.0,-0.07,0.62,40.09,XR,ATH,FINANCE
13,BSOFT,831.70,-2.65,-21.74,92.68,50.79,102014.0,-30582.0,110071.0,0.50,52.0,H-SC,10.71,171.0,-0.30,0.71,28.16,XR,ATH,IT
72,SURYODAY,216.00,0.53,-17.41,48.98,23.03,72437.0,-31181.0,147890.0,60.09,59.0,H-SC,5.07,165.0,-0.43,0.95,46.74,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.07,-13.26,15.32,0.02,23195.0,-23155.0,151404.0,131.03,58.0,M-SC,7.79,240.0,-1.00,0.97,31.70,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-2.87,-28.85,58.68,12.90,91223.0,-63037.0,155459.0,129.57,64.0,H-SC,10.52,173.0,-0.69,1.00,29.26,OX40N,NTT,FINANCE
75,TATAELXSI,9161.00,-2.33,-17.61,60.26,32.03,65451.0,-23221.0,108614.0,-10.83,66.0,H-SC,6.96,157.0,-0.35,0.70,21.66,OX40N,NTT,IT
17,COALINDIA,484.83,-1.48,4.57,14.01,19.23,22044.0,6882.0,157342.0,22.81,74.0,L-LC,6.58,182.0,0.31,1.01,26.82,XY25,ATH,MINING
85,VOLTAS,1856.00,-2.55,14.93,26.34,45.20,58044.0,28623.0,220365.0,3.76,63.0,X-MC,4.57,80.0,0.49,1.41,22.74,XY25,NTT,AC
36,ICICIPRULI,790.00,-0.47,12.75,15.95,30.73,35428.0,25115.0,222120.0,-13.58,70.0,X-MC,6.47,79.0,0.71,1.42,27.14,X40,ATH,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,21.74
1,25,44.76
2,50,75.46


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.33
LC    32.64
MC    28.05
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     24.19
X40      21.78
XY25     18.25
X40N     11.75
XR        8.31
AR        7.82
OX40N     7.03
SR        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-LC    29.49
X-MC    22.21
H-SC    21.03
M-SC    10.38
X-SC     7.20
H-MC     4.21
M-MC     1.31
M-LC     1.17
L-LC     1.01
H-LC     0.97
L-SC     0.72
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.59,-8.34,43.88
IT,11.73,-16.97,75.97
FINANCE,10.18,-12.43,59.61
REFINERIES,9.72,-5.93,32.73
MISC,6.61,-26.26,77.90
ELECTRICAL,5.39,-12.00,53.03
PAINTS,5.24,-12.85,29.61
INSURANCE,4.28,-1.55,36.30
PHARMA,3.61,-4.48,37.37


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3203405.0,21
AR,1353780.0,10
XR,1197445.0,12
X40,1124125.0,15
X40N,911181.0,9
XY25,855810.0,10
OX40N,696429.0,10
SR,281247.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3517699.0,24
X-MC,1569328.0,16
M-SC,1441495.0,15
X-LC,1386249.0,16
X-SC,799034.0,8
H-MC,423196.0,3
L-SC,178322.0,2
M-LC,109665.0,1
H-LC,83107.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1246043.0      6
           AR         917421.0      5
M-SC       XY24       831099.0      6
H-SC       XR         767965.0      7
X-LC       XY25       566658.0      6
X-MC       X40        507596.0      7
           XY24       441273.0      3
X-LC       X40        422826.0      6
X-MC       X40N       394389.0      4
X-SC       X40N       311755.0      3
H-SC       OX40N      305023.0      4
M-SC       OX40N      299640.0      5
X-SC       XY24       293576.0      3
H-SC       SR         281247.0      2
X-MC       XY25       226070.0      2
H-MC       AR         223510.0      2
X-LC       X40N       205037.0      2
H-MC       XY24       199686.0      1
X-SC       X40        193703.0      2
X-LC       XY24       191728.0      2
M-SC       XR         181014.0      2
           AR         129742.0      2
M-LC       XR         109665.0      1
L-SC       OX40N       91766.0      1
           XR          86556.0      1
H-LC       AR          83107.0      1
L-MC       XR          52245.0      1
M-MC       XY25        41038.0      1
L-LC       XY25        22044.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
